# Fast Food Nutritional Database: Loading Notebook

In [2]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import create_engine, inspect
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Create the engine
engine = create_engine("sqlite:///Tables/fastfood_nutritional_info.db")

In [4]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
# We can view all of the classes that automap found
Base.classes.keys()

['categories', 'menu_items', 'nutrition', 'restaurants']

In [6]:
#Save references to each table
Categories = Base.classes.categories
Menu_items = Base.classes.menu_items
Restaurants = Base.classes.restaurants
Nutrition = Base.classes.nutrition

In [7]:
# Create a session
session = Session(engine)

In [8]:
# Create a inspector
inspector = inspect(engine)

In [56]:
# Inspect Nurition
Nutrition_columns = inspector.get_columns('Nutrition')
for column in Nutrition_columns:
    print(column['name'], column["type"])

restaurant_id INTEGER
category_id INTEGER
item_id INTEGER
serving_size_oz INTEGER
calories INTEGER
calories_from_fat INTEGER
total_fat_g INTEGER
total_fat_%_dv INTEGER
saturated_fat_g INTEGER
saturated_fat_%_dv INTEGER
trans_fat_g INTEGER
cholesterol_mg INTEGER
cholesterol_%_dv INTEGER
sodium_mg INTEGER
sodium_%_dv INTEGER
carbohydrates_g INTEGER
carbohydrates_%_dv INTEGER
dietary_fiber_g INTEGER
dietary_fiber_% _dv INTEGER
sugars_g INTEGER
protein_g INTEGER
vitamin_a_%_dv INTEGER
vitamin_c_%_dv INTEGER
calcium_%_dv INTEGER
iron_%_dv INTEGER


In [57]:
# Inspect Restaurants
Restaurants_columns = inspector.get_columns('Restaurants')
for column in Restaurants_columns:
    print(column['name'], column["type"])

id INTEGER
name TEXT


In [58]:
# Inspect Categories
Categories_columns = inspector.get_columns('Categories')
for column in Categories_columns:
    print(column['name'], column["type"])

category_name TEXT
id INTEGER


In [59]:
# Inspect Menu Items
Items_columns = inspector.get_columns('Menu_items')
for column in Items_columns:
    print(column['name'], column["type"])

item_name TEXT
id INTEGER


# Test Queries

In [44]:
# Query the nutrition table - list the average serving size for menu items in the database
nutrition_test = session.query(func.avg(Nutrition.serving_size_oz)).all()

print(f"The average serving size for a menu item (ounces) is {nutrition_test}.")

The average serving size for a menu item (ounces) is [(10.612996131645557,)].


In [14]:
# Query the restaurants table - list restaurant id numbers and names
restaurants_test = session.query(Restaurants.id, Restaurants.name).order_by(Restaurants.id.desc()).all()
restaurants_test

[(3000, 'starbucks'), (2000, 'subway'), (1000, 'mcdonalds')]

In [16]:
# Query the categories table - how many food categories are there in the database?
categories_test = session.query(func.count(Categories.category_name)).all()

print(f"There are {categories_test} food categories in the database.")

There are [(31,)] food categories in the database.


In [17]:
# Query the menu items table - find the second menu item in the database
items_test = session.query(Menu_items.item_name).all()

print(f"The second menu item in the database is {items_test[1]}.")

The second menu item in the database is ('8-Grain Roll',).


## Queries joining together multiple tables

In [43]:
# Find the average fat content for salads (Categories + Nutrition tables) in the database

sel = [Categories.category_name, func.avg(Nutrition.total_fat_g)]

fat = session.query(*sel).filter(Nutrition.category_id==Categories.id).\
filter(Categories.category_name=="Salad").all()

print(f"Average fat(g): {fat}.")

Average fat(g): [('Salad', 14.61111111111111)].


In [35]:
# What are the average calories for menu items at each restaurant? (restaurants + nutrition table)

sel = [Restaurants.name, Nutrition.restaurant_id, func.avg(Nutrition.calories)]

avg_calories = session.query(*sel).filter(Nutrition.restaurant_id==Restaurants.id).\
group_by(Restaurants.name).\
order_by(func.avg(Nutrition.calories).desc()).all()

avg_calories

[('mcdonalds', 1000, 368.2692307692308),
 ('subway', 2000, 289.962962962963),
 ('starbucks', 3000, 245.68169014084506)]

In [8]:
# What are the items that have the highest calories (restaurants + menu_items + nutrition table)

sel = [Restaurants.name, Menu_items.item_name, Nutrition.calories]

calories = session.query(*sel).filter(Nutrition.restaurant_id==Restaurants.id).\
filter(Nutrition.item_id==Menu_items.id).\
order_by(Nutrition.calories.desc()).all()

calories

[('mcdonalds', 'Chicken McNuggets (40 piece)', 1880),
 ('subway', 'Chicken & Bacon Ranch Melt-Wrap', 1590),
 ('mcdonalds', 'Big Breakfast with Hotcakes (Large Biscuit)', 1150),
 ('mcdonalds', 'Big Breakfast with Hotcakes (Regular Biscuit)', 1090),
 ('subway', 'BBQ Rib Signature Wrap-Wrap', 1080),
 ('mcdonalds',
  'Big Breakfast with Hotcakes and Egg Whites (Large Biscuit)',
  1050),
 ('mcdonalds',
  'Big Breakfast with Hotcakes and Egg Whites (Regular Biscuit)',
  990),
 ('mcdonalds', 'Chicken McNuggets (20 piece)', 940),
 ('mcdonalds', 'McFlurry with M&M’s Candies (Medium)', 930),
 ('mcdonalds', 'Strawberry Shake (Large)', 850),
 ('mcdonalds', 'Chocolate Shake (Large)', 850),
 ('mcdonalds', 'Vanilla Shake (Large)', 820),
 ('mcdonalds', 'Shamrock Shake (Large)', 820),
 ('subway', 'Cripsy Cauliflower Ranch Signature Wrap-Wrap', 820),
 ('subway', 'Spicy Italian-Wrap', 820),
 ('subway', 'Tuna-Wrap', 820),
 ('mcdonalds', "McFlurry with Reese's Peanut Butter Cups (Medium)", 810),
 ('mcdonal

In [25]:
# Find breakfast items with more than 400 calories at each restaurant 
# (restaurants + menu_items + nutrition tables)

sel = [Restaurants.name, Menu_items.item_name, Nutrition.restaurant_id, Nutrition.calories]

breakfast_calories = session.query(*sel).filter(Nutrition.restaurant_id==Restaurants.id).\
filter(Nutrition.category_id==Categories.id).\
filter(Nutrition.item_id==Menu_items.id).\
filter(Categories.category_name=="Breakfast").\
filter(Nutrition.calories > 400).\
order_by(Nutrition.calories.desc()).all()

breakfast_calories

[('mcdonalds', 'Big Breakfast with Hotcakes (Large Biscuit)', 1000, 1150),
 ('mcdonalds', 'Big Breakfast with Hotcakes (Regular Biscuit)', 1000, 1090),
 ('mcdonalds',
  'Big Breakfast with Hotcakes and Egg Whites (Large Biscuit)',
  1000,
  1050),
 ('mcdonalds',
  'Big Breakfast with Hotcakes and Egg Whites (Regular Biscuit)',
  1000,
  990),
 ('mcdonalds', 'Big Breakfast (Large Biscuit)', 1000, 800),
 ('mcdonalds', 'Big Breakfast (Regular Biscuit)', 1000, 740),
 ('mcdonalds', 'Big Breakfast with Egg Whites (Large Biscuit)', 1000, 690),
 ('mcdonalds', 'Steak, Egg & Cheese Bagel', 1000, 670),
 ('mcdonalds', 'Big Breakfast with Egg Whites (Regular Biscuit)', 1000, 640),
 ('mcdonalds', 'Bacon, Egg & Cheese Bagel', 1000, 620),
 ('mcdonalds', 'Sausage Biscuit with Egg (Large Biscuit)', 1000, 570),
 ('mcdonalds', 'Bacon, Egg & Cheese Bagel with Egg Whites', 1000, 570),
 ('mcdonalds', 'Sausage, Egg & Cheese McGriddles', 1000, 550),
 ('mcdonalds', 'Steak & Egg Biscuit (Regular Biscuit)', 1000,

# Exploratory Plots

In [ ]:
# Saving to DataFrame
fastfood_df = pd.DataFrame(precip_data, columns = ['Date', 'Precipitation'])

precip_df.describe()